# Comparing Jellyfish Topologies

The following the generates Jellyfish topologies in three different ways (random, incremental, bipartite) and compares the throughput of two differnt types of traffic (all-to-all, random).


In [0]:
import networkx as nx
from random import sample, choice
from pathlib import Path
import pandas as pd

# define graphs to generate and test
graphs = [(64, 8), (100, 8), (200, 8), (100, 12), (200, 12), (300, 12)]

def update_progress(progress):
  """Very simple progress notifiaction function"""
  print(f"[{progress}/100")

In [0]:
def generate_jellyfish_random(nodes, degree):
  """Generate a Jellyfish Graph randomly"""
  # create empty graph with n nodes
  G = nx.empty_graph(nodes)
  # iterate until all switch ports are taken
  while sum([ degree for node, degree in G.degree]) < (nodes - 1) * degree:
    # pick two nodes randomly
    node_u, node_v = sample(G.nodes(), 2)
    # check if port available
    if G.degree[node_u] < degree and G.degree[node_v] < degree: 
      # check if not already connected
      if not G.has_edge(node_u, node_v):
        # connect nodes
        G.add_edge(node_u, node_v)

  # the second part runs until every node has at least a degree of d - 1
  while min(G.degree, key = lambda t: t[1])[1] <= degree - 2:
    node_x = min(G.degree, key = lambda t: t[1])[0]
    # add edges until node_x has degree
    while G.degree[node_x] < degree - 1:
      # select a random edge
      node_u, node_v = choice(list(G.edges()))

      # skip if node_x already connected
      if G.has_edge(node_x, node_u) or G.has_edge(node_x, node_v):
        continue

      # replace edge with two edges involving node_x
      G.remove_edge(node_u, node_v)
      G.add_edge(node_x, node_u)
      G.add_edge(node_x, node_v)

  return G

def generate_jellyfish_incremental(nodes, degree):
  """Generate a Jellyfish Graph incremental"""
  # generate fully connected graph with d + 1
  G = nx.complete_graph(degree + 1)

  # iterate over all missing nodes
  for node_x in range(degree + 1, nodes + 1):
    G.add_node(node_x)

    # add edges until node_x has degree
    while G.degree[node_x] < degree:
      # select a random edge
      node_u, node_v = choice(list(G.edges()))

      # skip if node_x already connected
      if G.has_edge(node_x, node_u) or G.has_edge(node_x, node_v):
        continue

      # replace edge with two edges involving node_x
      G.remove_edge(node_u, node_v)
      G.add_edge(node_x, node_u)
      G.add_edge(node_x, node_v)

  return G

def generate_jellyfish_bipartite(nodes, degree):
  """Generate a Jellyfish Graph bipartite"""
  # generate fully connected graph with d + 1
  G = nx.complete_bipartite_graph(degree, degree)

  # iterate over all missing nodes
  for node_x in range(degree * 2, nodes, 2):
    # get both halfs of the graph
    left, right = nx.bipartite.sets(G)

    # get random edges based on graph degree
    random_edges = sample(G.edges, degree)

    # add both nodes to graph
    G.add_node(node_x)
    G.add_node(node_x + 1)

    while G.degree[node_x] < degree:
      node_u, node_v = choice(list(G.edges()))
      if (G.has_edge(node_x, node_u) or G.has_edge(node_x, node_v) 
            or G.has_edge(node_x + 1, node_u) or G.has_edge(node_x + 1, node_v)):
        continue

      G.remove_edge(node_u, node_v)

      if node_u in left:
        if G.has_edge(node_x, node_u):
          print("k")
        G.add_edge(node_x, node_u)
        G.add_edge(node_x + 1, node_v)
      else:
        G.add_edge(node_x, node_v)
        G.add_edge(node_x + 1, node_u)

  return G

def create_all2all_traffic(G):
  """Create traffic from every node to every other node"""
  matrix = []
  for i in range(len(G.nodes)):
    matrix.append([])
    for j in range(len(G.nodes)):
      if i == j:
        matrix[i].append(0)
      else:
        matrix[i].append(1)

  output_lines = [
    ]
  for i in range(len(G.nodes)):
    for j in range(len(G.nodes)):
      if matrix[i][j] == 1:
        # add traffic of 1.0 between nodes
        output_lines.append((i, j, 1.0))
  return output_lines


def create_random_traffic(G):
  """Create random traffic

  Each node sends traffic to a random node and recieves from a random node
  """
  # store which nodes do not send/receive yet
  candidates_in = list(G.nodes)
  candidates_out = list(G.nodes)

  traffic = []
  
  # perform step based on number of nodes
  for i in range(len(G.nodes)):
    # get random node to receive and send
    node_u  = choice(candidates_in)
    node_v  = choice(candidates_out)

    # remove chosen nodes from candidates
    candidates_in.remove(node_u)
    candidates_out.remove(node_v)

    # create edge between nodes
    traffic.append((node_u, node_v))

  output_lines = []
  for u, v in sorted(traffic):
    # add traffic of 1.0 between nodes
    output_lines.append((u, v, 1.0))

  return output_lines


def calc_ecmp(G, paths, src, load, traffic_type):
  """Calculate ECMP routes and add traffic
  
  This function is implenented recursively to support branching of ECMP path. 
  The implementation takes advantage of the fact that each node only appears 
  at a specific position in the shortest path as they all share the same weight.
  """

  # the the destination based on last path element
  dest = paths[0][-1]

  # instantly return if destination is equal to source
  if src == dest:
    return

  # get path including the current source
  active_paths = list(filter(lambda p: src in p, paths))

  # get current position in path
  path_position = active_paths[0].index(src)

  # get next hops
  next_hops = set(map(lambda x: x[path_position + 1], active_paths))

  # skip if next hop is equal to destination
  if next_hops != { dest }:
    # iterate over all hops coming after current source
    for hop in next_hops:
      # divide the load by the number of found hops
      new_load = load/len(next_hops)

      # if edges `total_load` data is not yet set initialize it
      if f"total_load_{traffic_type}" not in G.edges[src, hop]:
        G.edges[src, hop][f"total_load_{traffic_type}"] = new_load
      else:
        # or add load to existing data
        G.edges[src, hop][f"total_load_{traffic_type}"] += new_load

      # recursive call of function for hop
      calc_ecmp(G, paths, hop, new_load, traffic_type)

def get_graph_stats(G):
  """Return graph stats, specifically the throughput `r`"""
  stats = []
  traffic_all2all = create_all2all_traffic(G)

  for u, v, load in traffic_all2all:
    paths = list(nx.all_shortest_paths(G, u, v, weight = "weight"))
    calc_ecmp(G, paths, u, load, "all2all")

  traffic_random = create_random_traffic(G)

  for u, v, load in traffic_random:
    paths = list(nx.all_shortest_paths(G, u, v, weight = "weight"))
    calc_ecmp(G, paths, u, load, "random")

  asp = nx.average_shortest_path_length(G)
  stats.append(asp)
  max_all2all = max(G.edges.data("total_load_all2all", default = 0.0), key = lambda x: x[2])[2]
  stats.append(max_all2all)
  stats.append(1/max_all2all)
  max_random = max(G.edges.data("total_load_random", default = 0.0), key = lambda x: x[2])[2]
  stats.append(max_random)
  stats.append(1/max_random)
  return stats

In [0]:
# This code fragment is only used for progress visualisation and based on
# https://www.mikulskibartosz.name/how-to-display-a-progress-bar-in-jupyter-notebook/
from IPython.display import clear_output

def update_progress(progress):
  bar_length = 20
  if isinstance(progress, int):
    progress = float(progress)
  if not isinstance(progress, float):
    progress = 0
  if progress < 0:
    progress = 0
  if progress >= 1:
    progress = 1

  block = int(round(bar_length * progress))

  clear_output(wait = True)
  text = "Progress: [{0}] {1:.1f}%".format( "#" * block + "-" * (bar_length - block), progress * 100)
  print(text)

In [12]:
data = []
runs = 5
counter = 0

print(f"Start graph generation for random/incremental/bipartite with average of {runs}")
for graph in graphs:
  for i in range(runs):
    nodes, degree = graph
    graph_data = ["Random", nodes, degree]
    G = generate_jellyfish_random(nodes, degree)
    graph_data.append(len(G.edges))
    graph_data.extend(get_graph_stats(G))
    data.append(graph_data)

    nodes, degree = graph
    graph_data = ["Incremental", nodes, degree]
    G = generate_jellyfish_incremental(nodes, degree)
    graph_data.append(len(G.edges))
    graph_data.extend(get_graph_stats(G))
    data.append(graph_data)

    nodes, degree = graph
    graph_data = ["Bipartite", nodes, degree]
    G = generate_jellyfish_bipartite(nodes, degree)
    graph_data.append(len(G.edges))
    graph_data.extend(get_graph_stats(G))
    data.append(graph_data)

    counter += 1
    update_progress(counter / (runs * len(graphs)))

Progress: [####################] 100.0%


In [13]:
# put all gathered data into a data frame
df = pd.DataFrame(data,columns=["Generator", 'Nodes', "Degree", "Edges",' average shortest path', "max all2all", "throughput all2all", "max random", "throughput random"])

# calculate the average for each generator and graph combination

df_formated = df.groupby(["Generator", "Nodes", "Degree"]).mean().sort_values(by=["Nodes", "Degree", "Generator"], ascending=[True, True, False])
display(df_formated)

,,,Edges,average shortest path,max all2all,throughput all2all,max random,throughput random
Generator,Nodes,Degree,,,,,,
Random,64,8,254.4,2.195040,30.704563,0.032877,1.506667,0.667121
Incremental,64,8,260.0,2.189231,29.787480,0.033679,1.918333,0.526897
Bipartite,64,8,256.0,2.341468,30.123651,0.033298,1.570238,0.641520
Random,100,8,398.0,2.430465,54.985675,0.018207,2.131250,0.478293
Incremental,100,8,404.0,2.415683,53.037798,0.018908,2.100000,0.478837
Bipartite,100,8,400.0,2.606545,57.800714,0.017317,1.867798,0.545837
Random,100,12,597.8,2.072808,28.762197,0.035017,1.608333,0.652721
Incremental,100,12,606.0,2.069861,26.939525,0.037287,1.965833,0.517575
Bipartite,100,12,600.0,2.293091,30.418822,0.033012,1.296582,0.778937
